# Unsupervised Predict Student Solution

© Explore Data Science Academy

---
### Honour Code

I {**CB3: Atlegang Mogane, Lebogang Gift Molepo, Mbalenhle Malinga, Mike Ngwenya,Alette Baloyi**}, confirm - by submitting this document - that the solutions in this notebook are a result of my own work and that I abide by the [EDSA honour code](https://drive.google.com/file/d/1QDCjGZJ8-FmJE3bZdIQNwnJyQKPhHZBn/view?usp=sharing).

Non-compliance with the honour code constitutes a material breach of contract.


### Predict Overview: Movie Recommendation 2022
In today’s technology driven world, recommender systems are socially and economically critical to ensure that individuals can make optimised choices surrounding the content they engage with on a daily basis. One application where this is especially true is movie recommendations; where intelligent algorithms can help viewers find great titles from tens of thousands of options.

With this context, EDSA is challenging you to construct a recommendation algorithm based on content or collaborative filtering, capable of accurately predicting how a user will rate a movie they have not yet viewed, based on their historical preferences.

Providing an accurate and robust solution to this challenge has immense economic potential, with users of the system being personalised recommendations - generating platform affinity for the streaming services which best facilitates their audience's viewing

<a id="cont"></a>

## Table of Contents
<a href=#one>1. Introduction</a>

<a href=#two>2. Problem Statement</a>

<a href=#three>3. Importing Packages</a>

<a href=#four>4. Loading Data</a>

<a href=#one>5. Exploratory Data Analysis (EDA)</a>



<a id="one"></a>
## 1. Introduction
<a href=#cont>Back to Table of Contents</a>

<a id="one"></a>
## 2. Problem Statement
<a href=#cont>Back to Table of Contents</a>

<a id="one"></a>
### 3. Importing Packages
<a href=#cont>Back to Table of Contents</a>

In [9]:
# Install packages
# Packages for data processiong
import numpy as np
import pandas as pd
import datetime
from sklearn import preprocessing
from sklearn.datasets import make_blobs
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfTransformer

import re
from scipy.sparse import csr_matrix
import scipy as sp

# Packages for visualization
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Packages for model evaluation
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from time import time

# Packages to avoid warnings
import warnings
warnings.filterwarnings("ignore")

# Packages for saving models
import pickle

# Ramdomization
import random

sns.set(style='whitegrid', palette='muted',
        rc={'figure.figsize': (15,10)})

In [ ]:
from sklearn.metrics.pairwise import cosine_????

<a id="one"></a>
### 4. Loading Data
<a href=#cont>Back to Table of Contents</a>

In [10]:
movies_df = pd.read_csv('./movies.csv')
imdb_df = pd.read_csv('./imdb_data.csv')
genome_score = pd.read_csv('./genome_scores.csv')
genome_tags = pd.read_csv('./genome_scores.csv')
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')
tags_df= pd.read_csv('./tags.csv')
links_df = pd.read_csv('./links.csv')
sample_submissiom_df = pd.read_csv('./sample_submission.csv')

In [11]:
train_df.head(4)

,userId,movieId,rating,timestamp
0,5163,57669,4.0,1518349992
1,106343,5,4.5,1206238739
2,146790,5459,5.0,1076215539
3,106362,32296,2.0,1423042565


In [12]:
train_df.shape

(10000038, 4)

<a id="one"></a>
### 5. EDA (Exploratory Data Analysis)
<a href=#cont>Back to Table of Contents</a>

In [14]:
train_df.shape

(10000038, 4)